In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:


def Fundamental_Mat_Estimation(img_left_points, img_right_points):
    X = []  # Inicializamos una lista vacía para almacenar las filas de la matriz A

    # Iteramos sobre los puntos de la imagen (de 0 a 7)
    for i in range(8):
        u_l, v_l = img_left_points[i]  # Coordenadas del punto izquierdo
        u_r, v_r = img_right_points[i]  # Coordenadas del punto derecho

        # Creamos una fila para la matriz A con los productos y coordenadas
        fila = np.asarray([u_l * u_r, u_l * v_r, u_l, v_l * u_r, v_r * v_l, v_l, u_r, v_r, 1])

        # Agregamos la fila a la lista A
        X.append(fila)

    # Convertimos la lista de filas en una matriz numpy
    X = np.asarray(X)

    # Calculamos la descomposición de valores singulares (SVD) de la matriz A
    U, S, V = np.linalg.svd(X)

    # Reorganizamos el último vector de V en una matriz 3x3 para obtener la matriz fundamental F
    F = np.reshape(V[-1, :], (3, 3))

    # Descomponemos F nuevamente usando SVD para hacer que la tercera singular value sea cero
    u, s, vt = np.linalg.svd(F)
    s = np.diag(s)
    s[2, 2] = 0

    # Reconstituimos F con la tercera singular value igual a cero
    F = u @ (s @ vt)

    return F  # Devolvemos la matriz fundamental estimada


In [ ]:

def RANSAC(Left_points, Right_points, iteraciones, tol):
    n = len(Left_points)  # Número de puntos
    Max_Inlier = 0  # Inicializamos el contador para el máximo número de inliers

    for i in range(iteraciones):
        Inlier = 0  # Inicializamos el contador de inliers para esta iteración

        # Seleccionamos aleatoriamente 8 pares de puntos
        Idx_random = np.random.choice(n, size=8, replace=False)
        Left_points_random = Left_points[Idx_random, :]
        Right_points_random = Right_points[Idx_random, :]

        # Estimamos la matriz fundamental F con estos 8 pares de puntos
        F = Fundamental_Mat_Estimation(Left_points_random, Right_points_random)

        for j in range(n):
            # Obtenemos un par de puntos
            Lpts, Rpts = Left_points[j], Right_points[j]

            # Convertimos los puntos en vectores homogéneos (añadiendo 1)
            Lpts, Rpts = np.append(Lpts, 1).reshape((3, 1)), np.append(Rpts, 1).reshape((3, 1))

            # Calculamos la distancia epipolar (residual)
            res = (Lpts.T @ F) @ Rpts

            # Comparamos la distancia epipolar con el umbral de tolerancia
            if abs(res) <= tol:
                Inlier += 1  # Si está dentro de la tolerancia, lo consideramos un inlier

        if Inlier > Max_Inlier:
            Max_Inlier = Inlier  # Actualizamos el número máximo de inliers
            Final_F = F  # Actualizamos la mejor matriz fundamental encontrada hasta ahora

    return Final_F  # Devolvemos la matriz fundamental final con el mayor número de inliers


In [ ]:
def drawEpipolarLines(img1,img2,lines,pts1,pts2):

    h,w = img1.shape[0],img1.shape[1] 
    nimg1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
    nimg2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for epipolar,pt1,pt2 in zip(lines,pts1,pts2):
    # Random color
        color = tuple(np.random.randint(0,255,3).tolist())
        x0,y0 = map(int, [0, -epipolar[2]/epipolar[1] ])
        x1,y1 = map(int, [w, -(epipolar[2]+epipolar[0]*w)/epipolar[1]])
        cv2.line(nimg1, (x0,y0), (x1,y1), color,1)
        cv2.circle(nimg1,pt1,5,color,-1)
        cv2.circle(nimg2,pt2,5,color,-1)
    return nimg1,nimg2

In [ ]:

def SIFT_Matches(image1, image2, MAX_FEATURES=3000):
    # Convertimos las imágenes de color a escala de grises
    new_image_1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    new_image_2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Creamos un detector SIFT con un límite máximo de características
    sift = cv2.SIFT_create(MAX_FEATURES)

    # Detectamos y calculamos descriptores para las características en ambas imágenes
    Key_Points_1, des1 = sift.detectAndCompute(new_image_1, None)
    Key_Points_2, des2 = sift.detectAndCompute(new_image_2, None)

    # Utilizamos el matcher de fuerza bruta (Brute-Force) para hacer coincidir los descriptores
    Bjf = cv2.BFMatcher()
    matches = Bjf.knnMatch(des1, des2, k=2)

    # Filtramos las coincidencias para mantener solo las que son lo suficientemente buenas
    Correct_Matches = []
    for m, n in matches:
        if m.distance < 0.8 * n.distance:
            Correct_Matches.append([m])

    # Extraemos las coordenadas de las características coincidentes en ambas imágenes
    left_coords = np.float32([Key_Points_1[m[0].queryIdx].pt for m in Correct_Matches])
    right_coords = np.float32([Key_Points_2[m[0].trainIdx].pt for m in Correct_Matches])

    return left_coords, right_coords


In [ ]:

#importamos imagenes 
img1 = cv2.imread('MIT2.jpg')
img2 = cv2.imread('MIT1.jpg')

left_pts, right_pts = SIFT_Matches(img1, img2)

F = RANSAC(left_pts, right_pts, 1500, 0.02)


In [ ]:
#Epilines corresponding to points in right image (second image) and
# drawing its lines on left image


img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
lines1 = cv2.computeCorrespondEpilines(right_pts.reshape(-1,1,2), 2,F)
lines1 = lines1.reshape(-1,3)
img5,img6 = drawEpipolarLines(img1,img2,lines1,left_pts.astype(int),right_pts.astype(int))
cv2.imwrite('Res1.jpg', img5)
cv2.imwrite('Res2.jpg', img6)

fig, ax=plt.subplots(1,2,dpi=216)
plt.subplot(121),plt.imshow(img5),plt.xticks([]),plt.yticks([])
plt.subplot(122),plt.imshow(img6),plt.xticks([]),plt.yticks([])
plt.show()

